In [104]:
import pandas as pd
import numpy as np

## Reading data

In [105]:
large = False
tournament = False

if tournament:
    tmt = "t_"
else:
    tmt = "l_"

if large:
    path = "market_value model\large"
else:
    path = "market_value model"

shooting = pd.read_csv(path + "/mv-simulation-shooting.csv", index_col=0)
passing = pd.read_csv(path + "/mv-simulation-passing.csv", index_col=0)
defense = pd.read_csv(path + "/mv-simulation-defense.csv", index_col=0)
goalkeeping = pd.read_csv(path + "/mv-simulation-goalkeeping.csv", index_col=0)

player_shooting = pd.read_csv(f"../data/processed-data/{tmt}shooting_clean.csv", index_col=0)
player_passing = pd.read_csv(f"../data/processed-data/{tmt}passing_clean_90.csv", index_col=0)
player_defense = pd.read_csv(f"../data/processed-data/{tmt}defense_clean.csv", index_col=0)
player_goalkeeping = pd.read_csv(f"../data/processed-data/{tmt}goalkeeping_clean.csv", index_col=0)

all_players = pd.concat([
    player_shooting[['Player', 'Nation', 'Squad', 'Age', 'Year', 'FW', 'MF', 'DF', 'GK']],
    player_passing[['Player', 'Nation', 'Squad', 'Age', 'Year', 'FW', 'MF', 'DF', 'GK']],
    player_defense[['Player', 'Nation', 'Squad', 'Age', 'Year', 'FW', 'MF', 'DF', 'GK']],
    player_goalkeeping[['Player', 'Nation', 'Squad', 'Age', 'Year', 'FW', 'MF', 'DF', 'GK']],
    ])

# all_players = pd.concat([
#     player_shooting[['Player', 'Nation', 'Age', 'Year', 'FW', 'MF', 'DF', 'GK']],
#     player_passing[['Player', 'Nation', 'Age', 'Year', 'FW', 'MF', 'DF', 'GK']],
#     player_defense[['Player', 'Nation', 'Age', 'Year', 'FW', 'MF', 'DF', 'GK']],
#     player_goalkeeping[['Player', 'Nation', 'Age', 'Year', 'FW', 'MF', 'DF', 'GK']],
#     ])

all_players = all_players[all_players.Year == 2021].drop_duplicates().reset_index(drop=True)
print(all_players.shape)
all_players = all_players.assign(Raritan = lambda x: x.Nation == "Rarita")

(2822, 9)


In [106]:
print(all_players[all_players[["Player", "Nation", "Age"]].duplicated()].sort_values(by = "Player").shape)

all_players_group = all_players.groupby(["Player", "Nation", "Age"])
all_players = all_players_group[['FW', 'MF', 'DF', 'GK', 'Raritan']].any().reset_index()
print(all_players.shape)

(105, 10)
(2717, 8)


In [107]:
tournament_players = pd.read_csv("../data/optim-data/0_tournament_players.csv", index_col=0)
print(tournament_players.shape)

(488, 8)


In [108]:
all_players = all_players[~all_players.Player.isin(tournament_players.Player)].reset_index(drop=True)


In [109]:
trials = max(shooting.trial) + 1
# trials = 1

# Remove duplicates in players
shooting = shooting.groupby(["Player", "Nation", "trial"])[["mv0", "mv1", "mv2", "mv3", "mv4", "mv5", "mv6", "mv7", "mv8", "mv9"]].median().reset_index()
passing = passing.groupby(["Player", "Nation", "trial"])[["mv0", "mv1", "mv2", "mv3", "mv4", "mv5", "mv6", "mv7", "mv8", "mv9"]].median().reset_index()
defense = defense.groupby(["Player", "Nation", "trial"])[["mv0", "mv1", "mv2", "mv3", "mv4", "mv5", "mv6", "mv7", "mv8", "mv9"]].median().reset_index()
goalkeeping = goalkeeping.groupby(["Player", "Nation", "trial"])[["mv0", "mv1", "mv2", "mv3", "mv4", "mv5", "mv6", "mv7", "mv8", "mv9"]].median().reset_index()

In [110]:
pd.options.display.max_rows = 1000
all_players[["FW","MF", "DF", "GK", "Raritan"]] = all_players[["FW","MF", "DF", "GK", "Raritan"]].astype(int)
if large:
    all_players.to_csv(r"..\data\optim-data\large\0_selection_players.csv")
else:
    all_players.to_csv(r"..\data\optim-data\0_selection_players.csv")
# all_players.to_csv(r"..\data\optim-data\0_tournament_players.csv")

In [111]:
all_players['player_id'] = all_players.apply(lambda row: "-".join([row[0], row[1]]), axis=1)

shooting['player_id'] = shooting.apply(lambda row: "-".join([row[0], row[1]]), axis=1)
passing['player_id'] = passing.apply(lambda row: "-".join([row[0], row[1]]), axis=1)
defense['player_id'] = defense.apply(lambda row: "-".join([row[0], row[1]]), axis=1)
goalkeeping['player_id'] = goalkeeping.apply(lambda row: "-".join([row[0], row[1]]), axis=1)

shooting.index = pd.MultiIndex.from_frame(shooting[['player_id', 'trial']])
passing.index = pd.MultiIndex.from_frame(passing[['player_id', 'trial']])
defense.index = pd.MultiIndex.from_frame(defense[['player_id', 'trial']])
goalkeeping.index = pd.MultiIndex.from_frame(goalkeeping[['player_id', 'trial']])

In [112]:
all_players.iloc[27, ]

Player                     a.fekete
Nation                Byasier Pujan
Age                              17
FW                                0
MF                                1
DF                                0
GK                                0
Raritan                           0
player_id    a.fekete-Byasier Pujan
Name: 27, dtype: object

In [113]:
from itertools import product

player_mv = pd.DataFrame()
player_mv.index = pd.MultiIndex.from_tuples(list(product(all_players.player_id, np.arange(trials))))

for i in all_players.index:
    if (i % 100 == 0):
        print(f"{i} out of {all_players.shape[0]}")

    player_id = "-".join([all_players.iloc[i, 0], all_players.iloc[i, 1]])
    
    role_FW = all_players.iloc[i, 3]
    role_MF = all_players.iloc[i, 4]
    role_DF = all_players.iloc[i, 5]
    role_GK = all_players.iloc[i, 6]

    best_role = -1
    mv = float("-Inf")
    best_df = None
    other_mv = float("-Inf")
    other_df = None
    
    for role_i, role in enumerate([role_FW, role_MF, role_DF, role_GK]):
        if role_i == 0:
            df = shooting
        elif role_i == 1:
            df = passing
        elif role_i == 2:
            df = defense
        elif role_i == 3:
            df = goalkeeping
        
        try:
            if role == 1:
                if best_role == -1 or df.loc[(player_id, 0), "mv0"] > mv:
                    best_role = role_i
                    best_df = df
                    mv = df.loc[(player_id, 0), "mv0"]
                if df.loc[(player_id, 0), "mv0"] > other_mv:
                    other_mv = df.loc[(player_id, 0), "mv0"]
                    other_df = df
                    
        except KeyError:
            pass
    
    if mv == float("-Inf"):
        best_df = other_df
    
    all_players.iloc[i, 3] = 0
    all_players.iloc[i, 4] = 0
    all_players.iloc[i, 5] = 0
    all_players.iloc[i, 6] = 0

    try:
        all_players.iloc[i, best_role+3] = 1
    except:
        pass

    for j in np.arange(trials):
        for t in np.arange(10):
            mv_t = "mv" + str(t)
            
            try:
                player_mv.loc[(player_id, j), mv_t] = best_df.loc[(player_id, j), mv_t]
            except:
                break

player_mv = player_mv.reset_index()

0 out of 2422
100 out of 2422
200 out of 2422
300 out of 2422
400 out of 2422
500 out of 2422
600 out of 2422
700 out of 2422
800 out of 2422
900 out of 2422
1000 out of 2422
1100 out of 2422
1200 out of 2422
1300 out of 2422
1400 out of 2422
1500 out of 2422
1600 out of 2422
1700 out of 2422
1800 out of 2422
1900 out of 2422
2000 out of 2422
2100 out of 2422
2200 out of 2422
2300 out of 2422
2400 out of 2422


In [117]:
all_players[2381:2385]

,Player,Nation,Age,FW,MF,DF,GK,Raritan,player_id
2381,z.knezevi,Rarita,28,1,0,0,0,1,z.knezevi-Rarita
2382,z.krajnc,Lefghau,29,0,0,1,0,0,z.krajnc-Lefghau
2383,z.lefevre,Quewenia,21,0,0,0,1,0,z.lefevre-Quewenia
2384,z.longole,Lefghau,24,0,0,1,0,0,z.longole-Lefghau


In [115]:
player_mv = player_mv.fillna(0)
player_mv = player_mv.rename(columns={"level_0": "player_id", "level_1": "trial"})
player_mv = player_mv.sort_values(by = ["trial", "player_id"])
if large:
    player_mv.to_csv(r"..\data\optim-data\large\0_selection_mv.csv")    
else:
    player_mv.to_csv(r"..\data\optim-data\0_selection_mv.csv")


# player_mv.to_csv(r"..\data\optim-data\0_tournament_mv.csv")

In [116]:
all_players.to_csv("../data/optim-data/0_selection_players.csv")